In [1]:
!pip install datasets transformers[sentencepiece]
!pip install accelerate

     |████████████████████████████████| 311 kB 5.3 MB/s 
     |████████████████████████████████| 3.5 MB 39.8 MB/s 
     |████████████████████████████████| 133 kB 46.7 MB/s 
     |████████████████████████████████| 1.1 MB 45.5 MB/s 
     |████████████████████████████████| 67 kB 5.0 MB/s 
     |████████████████████████████████| 243 kB 47.9 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
     |████████████████████████████████| 271 kB 45.7 MB/s 
     |████████████████████████████████| 144 kB 49.2 MB/s 
     |████████████████████████████████| 895 kB 49.0 MB/s 
     |████████████████████████████████| 6.8 MB 35.6 MB/s 
     |████████████████████████████████| 596 kB 46.2 MB/s 
     |████████████████████████████████| 1.2 MB 50.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 58 kB 3.1 MB/s 


In [2]:
from huggingface_hub import notebook_login

notebook_login()


Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [5]:

from datasets import load_dataset, load_metric

raw_datasets = load_dataset("json", data_files="small_data_DS_format.json")

Using custom data configuration default-8243a3620de5050a


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-8243a3620de5050a/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("json", data_files="small_data_DS_format.json")
split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=20)
split_datasets["validation"] = split_datasets.pop("test")

from transformers import pipeline

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
translator = pipeline("translation", model=model_checkpoint)

from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="tf")

max_input_length = 23
max_target_length = 23


def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)


from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])

Using custom data configuration default-8243a3620de5050a
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-8243a3620de5050a/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/287M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/784k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.28M [00:00<?, ?B/s]

  0%|          | 0/125 [00:00<?, ?ba/s]

  0%|          | 0/14 [00:00<?, ?ba/s]

In [7]:
!pip install sacrebleu

from datasets import load_metric

metric = load_metric("sacrebleu")

import numpy as np


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    # print("Return:- ", metric.compute(predictions=decoded_preds, references=decoded_labels))
    # print("decoded_preds:- ", decoded_preds)
    # print("decoded_labels:- ", decoded_labels)
    # print("Done")
    return metric.compute(predictions=decoded_preds, references=decoded_labels)




     |████████████████████████████████| 90 kB 4.1 MB/s eta 0:00:011


Downloading:   0%|          | 0.00/2.36k [00:00<?, ?B/s]

Using amp half precision backend
***** Running Evaluation *****
  Num examples = 13786
  Batch size = 64


Trainer is attempting to log a value of "[125424, 84533, 57443, 40353]" of type <class 'list'> for key "eval/counts" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[169320, 155534, 141748, 127962]" of type <class 'list'> for key "eval/totals" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[74.07512402551382, 54.350174238430185, 40.52473403504811, 31.535143245651053]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_bp': 0.9934012678391214,
 'eval_counts': [125424, 84533, 57443, 40353],
 'eval_loss': 0.9198387861251831,
 'eval_precisions': [74.07512402551382,
  54.350174238430185,
  40.52473403504811,
  31.535143245651053],
 'eval_ref_len': 170441,
 'eval_runtime': 558.0346,
 'eval_samples_per_second': 24.705,
 'eval_score': 47.312052300226874,
 'eval_steps_per_second': 0.387,
 'eval_sys_len': 169320,
 'eval_totals': [169320, 155534, 141748, 127962]}

In [9]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"marian-finetuned-kde4-en-to-fr",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)


from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# trainer.evaluate(max_length=max_target_length)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using amp half precision backend


In [10]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 124074
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3878


Step,Training Loss
500,0.075000
1000,0.055900
1500,0.045800


Step,Training Loss
500,0.075000
1000,0.055900
1500,0.045800
2000,0.040500
2500,0.036200
3000,0.032000
3500,0.029600


Saving model checkpoint to marian-finetuned-kde4-en-to-fr/checkpoint-3878
Configuration saved in marian-finetuned-kde4-en-to-fr/checkpoint-3878/config.json
Model weights saved in marian-finetuned-kde4-en-to-fr/checkpoint-3878/pytorch_model.bin
tokenizer config file saved in marian-finetuned-kde4-en-to-fr/checkpoint-3878/tokenizer_config.json
Special tokens file saved in marian-finetuned-kde4-en-to-fr/checkpoint-3878/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=3878, training_loss=0.043439604438045946, metrics={'train_runtime': 3932.7227, 'train_samples_per_second': 31.549, 'train_steps_per_second': 0.986, 'total_flos': 670206691639296.0, 'train_loss': 0.043439604438045946, 'epoch': 1.0})

In [11]:
trainer.evaluate(max_length=max_target_length)

***** Running Evaluation *****
  Num examples = 13786
  Batch size = 64


Trainer is attempting to log a value of "[168147, 152538, 137483, 122891]" of type <class 'list'> for key "eval/counts" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[170637, 156851, 143065, 129279]" of type <class 'list'> for key "eval/totals" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[98.54076196838903, 97.2502566129639, 96.09827700695489, 95.05874890740182]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'epoch': 1.0,
 'eval_bp': 1.0,
 'eval_counts': [168147, 152538, 137483, 122891],
 'eval_loss': 0.022329561412334442,
 'eval_precisions': [98.54076196838903,
  97.2502566129639,
  96.09827700695489,
  95.05874890740182],
 'eval_ref_len': 170441,
 'eval_runtime': 559.7025,
 'eval_samples_per_second': 24.631,
 'eval_score': 96.7283081766182,
 'eval_steps_per_second': 0.386,
 'eval_sys_len': 170637,
 'eval_totals': [170637, 156851, 143065, 129279]}

### Reference model (HF):- https://huggingface.co/course/chapter7/4?fw=pt

### Modifications:- 1. Dataset is different. Because HF reference was using large dataset and training time was very high due to high dimensionality. 2. HF reference has bug in compute_metrics() function and I have fixed it after lot of debugging.

### Only pre-trained model (without finetuning):- model was giving 47% BLEU score. After complete finetuning (I kept all parameters as it is), model gave 96% of BLEU score (on trainer.evaluate)

### Still, one can generate predictions and conduct error analysis for further optimisation.